In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/My Drive/Amazon Fine Food Review/Reviews.csv', delimiter = ',')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
df = df.dropna()
df.reset_index(inplace=True)

In [ ]:
X = df['Summary'][:100000]
type(X)

pandas.core.series.Series

In [ ]:
Y_ = df['Score'][:100000]
arr = np.empty(100000)
Y = pd.Series(arr)
for i in range (0,100000):
  if Y_[i] > 3:
    Y[i] = 1
  elif Y_[i]<3:
    Y[i] = -1
  else:
    Y[i] = 0
type(Y)

pandas.core.series.Series

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
voc_size=5000

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, 100000):
    review = re.sub('[^a-zA-Z]', ' ', X[i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[0]

'good qualiti dog food'

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[594, 1547, 125, 1780],
 [2940],
 [626, 4034],
 [2923, 314],
 [2144, 4527],
 [2384, 4527],
 [2144, 594, 1686, 222],
 [1062, 1596, 4527],
 [1516, 1732],
 [3273, 125, 1780],
 [4283, 322, 1588, 2868],
 [518, 539, 1831, 1780, 2583, 1440, 1780],
 [518, 1445, 4573, 1780],
 [4894, 3233],
 [784, 3284, 1887],
 [586, 3284, 2504],
 [3712, 4001],
 [539],
 [2144, 2197, 3333],
 [2970, 302, 4306],
 [4535, 4894],
 [3284],
 [1672, 1153],
 [3284],
 [4798, 2685, 4597],
 [3284, 784],
 [3987, 4297],
 [2144, 1965, 1586],
 [1887],
 [4283, 322, 1588, 2868],
 [2144, 922],
 [4001],
 [4283, 1456, 4737],
 [594, 1456],
 [2144, 1878, 4737, 352],
 [2529],
 [539, 4940, 508, 4737],
 [4737],
 [594, 1847, 1625, 1047],
 [3686, 3198, 2554],
 [3235, 4737, 1423, 4001, 2144],
 [4737, 4737, 4498],
 [1780, 2144],
 [594, 322, 2554],
 [2144, 4001, 2600],
 [4038, 4737],
 [594],
 [4699],
 [594, 3362, 1983, 1753, 2011, 4803],
 [1764],
 [4226],
 [322, 594, 1133, 3147],
 [3263, 2059, 91, 3779, 4820],
 [91, 3779],
 [116, 1557, 2004],

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1547  125 1780]
 [   0    0    0 ...    0    0 2940]
 [   0    0    0 ...    0  626 4034]
 ...
 [   0    0    0 ...    0    0 2650]
 [   0    0    0 ...    0 1887 1916]
 [   0    0    0 ...    0  594 4830]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  594, 1547,  125, 1780], dtype=int32)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_7 (LSTM)               (None, 100)               56400     
                                                                 
 dense_7 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=512)

Epoch 1/10
131/131 [==============================] - 22s 160ms/step - loss: -5.6209 - accuracy: 0.6964 - val_loss: -4.8391 - val_accuracy: 0.6769
Epoch 2/10
131/131 [==============================] - 21s 157ms/step - loss: -6.7025 - accuracy: 0.6956 - val_loss: -5.3511 - val_accuracy: 0.6829
Epoch 3/10
131/131 [==============================] - 21s 159ms/step - loss: -7.5818 - accuracy: 0.6987 - val_loss: -5.7083 - val_accuracy: 0.6805
Epoch 4/10
131/131 [==============================] - 20s 153ms/step - loss: -8.3748 - accuracy: 0.6985 - val_loss: -6.1868 - val_accuracy: 0.6908
Epoch 5/10
131/131 [==============================] - 20s 156ms/step - loss: -9.1736 - accuracy: 0.7006 - val_loss: -6.5172 - val_accuracy: 0.6882
Epoch 6/10
131/131 [==============================] - 22s 165ms/step - loss: -9.9705 - accuracy: 0.7033 - val_loss: -6.9678 - val_accuracy: 0.6898
Epoch 7/10
131/131 [==============================] - 21s 157ms/step - loss: -10.7359 - accuracy: 0.7070 - val_loss: -